## 读取数据集

In [1]:
from torchvision.datasets import CIFAR10 #下载图片数据集
from torch.utils.data import DataLoader #读取批次
import torchvision.transforms as transforms #张量转换
from torchvision.utils import save_image #保存图片

In [2]:
#读取数据集
dataset = CIFAR10(root = './data', 
                 download = True, transform = transforms.ToTensor()) #下载数据集
dataloader = DataLoader(dataset, batch_size= 64, shuffle= True) #按批次读取数据(一批64张，总共有50000张，所以有50000/64=781批)，shuffle= True打乱数据

Files already downloaded and verified


In [3]:
for batch_idx, data in enumerate(dataloader):
    real_images, _ = data
    batch_size = real_images.shape[0]
    print('#{} has {} images.'.format(batch_idx, batch_size))
    if batch_idx % 100 == 0:
        path = './data/CIFAR10_shuffled_batch{:03d}.png'.format(batch_idx)
        save_image(real_images, path, nrow=8, normalize=False)

#0 has 64 images.
#1 has 64 images.
#2 has 64 images.
#3 has 64 images.
#4 has 64 images.
#5 has 64 images.
#6 has 64 images.
#7 has 64 images.
#8 has 64 images.
#9 has 64 images.
#10 has 64 images.
#11 has 64 images.
#12 has 64 images.
#13 has 64 images.
#14 has 64 images.
#15 has 64 images.
#16 has 64 images.
#17 has 64 images.
#18 has 64 images.
#19 has 64 images.
#20 has 64 images.
#21 has 64 images.
#22 has 64 images.
#23 has 64 images.
#24 has 64 images.
#25 has 64 images.
#26 has 64 images.
#27 has 64 images.
#28 has 64 images.
#29 has 64 images.
#30 has 64 images.
#31 has 64 images.
#32 has 64 images.
#33 has 64 images.
#34 has 64 images.
#35 has 64 images.
#36 has 64 images.
#37 has 64 images.
#38 has 64 images.
#39 has 64 images.
#40 has 64 images.
#41 has 64 images.
#42 has 64 images.
#43 has 64 images.
#44 has 64 images.
#45 has 64 images.
#46 has 64 images.
#47 has 64 images.
#48 has 64 images.
#49 has 64 images.
#50 has 64 images.
#51 has 64 images.
#52 has 64 images.
#53

In [4]:
#对上面for循环的解释：
#batch_idx 是第几批，batch_size是一批有几个（64个，最后一批凑不齐只有16个）也可以用real_images.size(0)来表示。
#save_image是保存图片，使用if条件语句，每100批保存当批的图片，共有781批，所以会保存6张图。每一张图有64张小图，设置显示行数为8，就会有8*8=64张小图凑成的一张大图，normallize设置是否调整输张量的范围到[0,1]

## 设置网络

In [3]:
import torch.nn as nn

In [4]:
#构建鉴别网络
n_d_feature = 64 #潜在大小
n_channel = 3 #输入通道数
dnet = nn.Sequential(
        nn.Conv2d(n_channel, n_d_feature, kernel_size=4,
                 stride=2, padding=1),
        nn.LeakyReLU(0.2),
        nn.Conv2d(n_d_feature, 2 * n_d_feature, kernel_size=4,
                 stride=2, padding=1, bias=False),
        nn.BatchNorm2d(2 * n_d_feature),
        nn.LeakyReLU(0.2),
        nn.Conv2d(2 * n_d_feature, 4 * n_d_feature, kernel_size=4,
                 stride=2, padding=1, bias=False),
        nn.BatchNorm2d(4 * n_d_feature),
        nn.LeakyReLU(0.2),
        nn.Conv2d(4 * n_d_feature, 1, kernel_size=4))
print(dnet)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (1): LeakyReLU(negative_slope=0.2)
  (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (4): LeakyReLU(negative_slope=0.2)
  (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (7): LeakyReLU(negative_slope=0.2)
  (8): Conv2d(256, 1, kernel_size=(4, 4), stride=(1, 1))
)


In [5]:
#构建生成网络
latent_size = 64 #输入通道数
n_g_feature = 64 #输出通道数

gnet = nn.Sequential(
        nn.ConvTranspose2d(latent_size, 4 * n_g_feature, kernel_size=4,
                          bias=False),
        nn.BatchNorm2d(4 * n_g_feature),
        nn.ReLU(),
        nn.ConvTranspose2d(4 * n_g_feature, 2 * n_g_feature, kernel_size=4,
                          stride=2, padding=1, bias=False),
        nn.BatchNorm2d(2 * n_g_feature),
        nn.ReLU(),
        nn.ConvTranspose2d(2 * n_g_feature, n_g_feature, kernel_size=4,
                          stride=2, padding=1, bias=False),
        nn.BatchNorm2d(n_g_feature),
        nn.ReLU(),
        nn.ConvTranspose2d(n_g_feature, n_channel, kernel_size=4,
                          stride=2, padding=1),
        nn.Sigmoid())
print(gnet)

Sequential(
  (0): ConvTranspose2d(64, 256, kernel_size=(4, 4), stride=(1, 1), bias=False)
  (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): ReLU()
  (6): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (8): ReLU()
  (9): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (10): Sigmoid()
)


In [6]:
#网络初始化
import torch.nn.init as init
def weights_init(m):
    if type(m) in [nn.ConvTranspose2d, nn.Conv2d]:
        init.xavier_normal_(m.weight)
    elif type(m) == nn.BatchNorm2d:
        init.normal_(m.weight, 1.0, 0.02)
        init.constant_(m.bias, 0)
        
gnet.apply(weights_init)
dnet.apply(weights_init)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (1): LeakyReLU(negative_slope=0.2)
  (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (4): LeakyReLU(negative_slope=0.2)
  (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (7): LeakyReLU(negative_slope=0.2)
  (8): Conv2d(256, 1, kernel_size=(4, 4), stride=(1, 1))
)

## 训练网络

In [7]:
import torch
import torch.optim

In [8]:
#载入cpu训练的预参数
checkpoint_d = torch.load('discriminator_1.pth', map_location='cpu')
checkpoint_g = torch.load('generator_1.pth', map_location='cpu')
dnet.load_state_dict(checkpoint_d)
gnet.load_state_dict(checkpoint_g)

In [9]:
#定义损失
criterion = nn.BCEWithLogitsLoss()
#定义优化器
goptimizer = torch.optim.Adam(gnet.parameters(),
                             lr=0.0002, betas=(0.5, 0.999))
doptimizer = torch.optim.Adam(dnet.parameters(),
                             lr=0.0002, betas=(0.5, 0.999))

In [10]:
#生成噪音数据，输入到G网络的数据
batch_size = 64
fixed_noises = torch.randn(batch_size, latent_size, 1, 1)

In [11]:
#开始训练
epoch_num = 5#共训练10个周期
for epoch in range(epoch_num):
    for batch_idx, data in enumerate(dataloader):
        real_images, _ = data
        batch_size = real_images.shape[0]
        #训练判别器D
        labels = torch.ones(batch_size)#真实数据的标签：1
        preds = dnet(real_images) #将真实数据喂给D网络
        outputs = preds.reshape(-1) #转换成未知行
        dloss_real = criterion(outputs, labels)
        dmean_real = outputs.sigmoid().mean() #计算判别器将多少真数据判别为真，仅用于输出显示
        
        noises = torch.randn(batch_size, latent_size, 1, 1)
        fake_images = gnet(noises) #生成假数据
        labels = torch.zeros(batch_size)#生成假数据的标签：0
        fake = fake_images.detach() #类似于固定生成器参数
        preds = dnet(fake) #将假数据喂给判别器
        outputs = preds.reshape(-1)#转换成未知行
        dloss_fake = criterion(outputs, labels)
        dmean_fake = outputs.sigmoid().mean() #计算判别器将多少假数据判断为真，仅用于输出显示
        
        dloss = dloss_real + dloss_fake #总的鉴别器损失为两者之和
        dnet.zero_grad()#梯度清零
        dloss.backward()#反向传播
        doptimizer.step()
        
        #训练生成器G
        labels = torch.ones(batch_size)#在训练生成器G时，希望生成器的标签为1
        preds = dnet(fake_images)#让假数据通过鉴别网络
        outputs = preds.reshape(-1)#转换成未知行
        gloss = criterion(outputs, labels)
        gmean_fake = outputs.sigmoid().mean() #计算判别器将多少假数据判断为真，仅用于输出显示
        
        gnet.zero_grad()#梯度清零
        gloss.backward()#反向传播
        goptimizer.step()
        
        #输出本步训练结果
        print('[{}/{}]'.format(epoch, epoch_num) + '[{}/{}]'.format(batch_idx, len(dataloader)) +
             '鉴别器G损失:{:g} 生成器D损失：{:g}'.format(dloss, gloss) + 
             '真数据判真比例：{:g} 假数据判真比例：{:g}/{:g}'.format(dmean_real, dmean_fake, gmean_fake))
        if batch_idx % 100 == 0:
            fake = gnet(fixed_noises) #噪声生成假数据
            path = 'D:/pythonwork/pytorch专栏/神经网络与pytorch实战/pytorch-book-master/chapter10_GAN/data/images_epoch{:02d}_batch{:03d}.png'.format(epoch, batch_idx)
            save_image(fake, path, normalize=False)

[0/5][0/782]鉴别器G损失:0.111558 生成器D损失：6.83171真数据判真比例：0.955532 假数据判真比例：0.053147/0.00147781
[0/5][1/782]鉴别器G损失:0.38044 生成器D损失：1.62073真数据判真比例：0.718958 假数据判真比例：0.00520151/0.265323
[0/5][2/782]鉴别器G损失:0.681458 生成器D损失：4.847真数据判真比例：0.99477 假数据判真比例：0.41908/0.0113214


KeyboardInterrupt: 

## 保存模型

In [85]:
torch.save(gnet.state_dict(),'./generator.pth')
torch.save(dnet.state_dict(),'./discriminator.pth')